# Chapter 10: Distribution Ray Tracing

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/computer-vision/blob/main/chapter_10_distribution_ray_tracing.ipynb)

**Distribution ray tracing** extends basic ray tracing by sampling multiple rays per pixel to simulate realistic optical effects that cannot be achieved with single-ray sampling. This technique enables soft shadows, depth of field, motion blur, glossy reflections, and anti-aliasing.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
from typing import Tuple, List, Optional
from dataclasses import dataclass

In [ ]:
class Vec3:
    """3D Vector class"""
    def __init__(self, x: float = 0.0, y: float = 0.0, z: float = 0.0):
        self.x = x
        self.y = y
        self.z = z
    
    def __add__(self, other):
        return Vec3(self.x + other.x, self.y + other.y, self.z + other.z)
    
    def __sub__(self, other):
        return Vec3(self.x - other.x, self.y - other.y, self.z - other.z)
    
    def __mul__(self, scalar: float):
        return Vec3(self.x * scalar, self.y * scalar, self.z * scalar)
    
    def __truediv__(self, scalar: float):
        return Vec3(self.x / scalar, self.y / scalar, self.z / scalar)
    
    def __neg__(self):
        return Vec3(-self.x, -self.y, -self.z)
    
    def dot(self, other) -> float:
        return self.x * other.x + self.y * other.y + self.z * other.z
    
    def cross(self, other):
        return Vec3(
            self.y * other.z - self.z * other.y,
            self.z * other.x - self.x * other.z,
            self.x * other.y - self.y * other.x
        )
    
    def length(self) -> float:
        return math.sqrt(self.dot(self))
    
    def normalize(self):
        l = self.length()
        if l > 0:
            return self / l
        return Vec3(0, 0, 0)
    
    def __repr__(self):
        return f"Vec3({self.x:.3f}, {self.y:.3f}, {self.z:.3f})"

class Color:
    """RGB Color class"""
    def __init__(self, r: float = 0.0, g: float = 0.0, b: float = 0.0):
        self.r = r
        self.g = g
        self.b = b
    
    def __add__(self, other):
        return Color(self.r + other.r, self.g + other.g, self.b + other.b)
    
    def __mul__(self, other):
        if isinstance(other, Color):
            return Color(self.r * other.r, self.g * other.g, self.b * other.b)
        else:
            return Color(self.r * other, self.g * other, self.b * other)
    
    def __truediv__(self, scalar: float):
        return Color(self.r / scalar, self.g / scalar, self.b / scalar)
    
    def clamp(self, min_val: float = 0.0, max_val: float = 1.0):
        return Color(
            max(min_val, min(max_val, self.r)),
            max(min_val, min(max_val, self.g)),
            max(min_val, min(max_val, self.b))
        )
    
    def to_tuple(self) -> Tuple[float, float, float]:
        return (self.r, self.g, self.b)

class Ray:
    """Ray class with origin and direction"""
    def __init__(self, origin: Vec3, direction: Vec3, time: float = 0.0):
        self.origin = origin
        self.direction = direction.normalize()
        self.time = time  # For motion blur
    
    def at(self, t: float) -> Vec3:
        return self.origin + self.direction * t

print("✓ Base classes loaded")

## 1. Sampling Theory

**Problem:** Single-ray sampling creates aliasing artifacts and cannot capture distributed optical phenomena.

**Solution:** Sample multiple rays per pixel and average the results.

### Monte Carlo Integration

The rendering equation can be approximated using Monte Carlo integration:

$$
L_o(\mathbf{p}, \omega_o) \approx \frac{1}{N} \sum_{i=1}^{N} f(\mathbf{x}_i)
$$

where:
- $N$ is the number of samples
- $\mathbf{x}_i$ are random samples
- $f(\mathbf{x}_i)$ is the function being integrated

### Sampling Patterns

**Random Sampling:**
$$
\mathbf{x}_i = (u_i, v_i), \quad u_i, v_i \sim U(0,1)
$$

**Stratified Sampling (Jittered):**
$$
\mathbf{x}_{ij} = \left(\frac{i + \xi_1}{n}, \frac{j + \xi_2}{m}\right), \quad \xi_1, \xi_2 \sim U(0,1)
$$

Stratified sampling reduces variance and produces better convergence.

In [ ]:
def random_in_unit_disk() -> Tuple[float, float]:
    """Generate random point in unit disk using rejection sampling"""
    while True:
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)
        if x*x + y*y < 1:
            return (x, y)

def random_in_unit_sphere() -> Vec3:
    """Generate random point in unit sphere using rejection sampling"""
    while True:
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)
        z = random.uniform(-1, 1)
        if x*x + y*y + z*z < 1:
            return Vec3(x, y, z)

def random_unit_vector() -> Vec3:
    """Generate random unit vector on sphere surface"""
    return random_in_unit_sphere().normalize()

def random_in_hemisphere(normal: Vec3) -> Vec3:
    """Generate random vector in hemisphere around normal"""
    in_unit_sphere = random_in_unit_sphere()
    if in_unit_sphere.dot(normal) > 0.0:
        return in_unit_sphere
    else:
        return -in_unit_sphere

def stratified_sample_2d(n: int) -> List[Tuple[float, float]]:
    """Generate stratified samples in 2D (jittered grid)"""
    samples = []
    inv_n = 1.0 / n
    for i in range(n):
        for j in range(n):
            u = (i + random.random()) * inv_n
            v = (j + random.random()) * inv_n
            samples.append((u, v))
    return samples

print("✓ Sampling functions loaded")

## 2. Anti-Aliasing

**Problem:** Sampling one ray per pixel creates jagged edges (aliasing).

**Solution:** Cast multiple rays per pixel with slight position offsets.

### Super-Sampling Anti-Aliasing (SSAA)

For pixel $(i, j)$, generate $N$ samples:

$$
\mathbf{r}_k = \mathbf{o} + \left(i + u_k\right) \mathbf{dx} + \left(j + v_k\right) \mathbf{dy}
$$

where $u_k, v_k \in [0, 1)$ are random offsets.

Final pixel color:
$$
C(i,j) = \frac{1}{N} \sum_{k=1}^{N} L(\mathbf{r}_k)
$$

In [ ]:
class Camera:
    """Camera with depth of field support"""
    def __init__(self,
                 look_from: Vec3,
                 look_at: Vec3,
                 up: Vec3,
                 vfov: float,
                 aspect_ratio: float,
                 aperture: float = 0.0,
                 focus_distance: float = 1.0):
        
        self.origin = look_from
        self.lens_radius = aperture / 2.0
        
        theta = vfov * math.pi / 180.0
        h = math.tan(theta / 2.0)
        viewport_height = 2.0 * h
        viewport_width = aspect_ratio * viewport_height
        
        self.w = (look_from - look_at).normalize()
        self.u = up.cross(self.w).normalize()
        self.v = self.w.cross(self.u)
        
        self.horizontal = self.u * (viewport_width * focus_distance)
        self.vertical = self.v * (viewport_height * focus_distance)
        self.lower_left_corner = self.origin - self.horizontal / 2.0 - self.vertical / 2.0 - self.w * focus_distance
    
    def get_ray(self, s: float, t: float, time: float = 0.0) -> Ray:
        """Get ray for pixel coordinates (s, t) with depth of field"""
        if self.lens_radius > 0:
            rd_x, rd_y = random_in_unit_disk()
            rd = rd_x * self.lens_radius, rd_y * self.lens_radius
            offset = self.u * rd[0] + self.v * rd[1]
        else:
            offset = Vec3(0, 0, 0)
        
        direction = self.lower_left_corner + self.horizontal * s + self.vertical * t - self.origin - offset
        return Ray(self.origin + offset, direction, time)

print("✓ Camera class loaded")

## 3. Soft Shadows (Area Lights)

**Problem:** Point lights create hard shadows with sharp edges.

**Solution:** Use area lights and cast multiple shadow rays to different points on the light surface.

### Area Light Visibility

The visibility from point $\mathbf{p}$ to area light $A$ is:

$$
V(\mathbf{p}, A) = \frac{1}{N} \sum_{i=1}^{N} V(\mathbf{p}, \mathbf{l}_i)
$$

where:
- $\mathbf{l}_i$ are random points sampled on light surface
- $V(\mathbf{p}, \mathbf{l}_i) = 1$ if no occlusion, $0$ if occluded

### Soft Shadow Contribution

$$
L_d = k_d \cdot I \cdot V(\mathbf{p}, A) \cdot \max(0, \mathbf{n} \cdot \mathbf{l})
$$

In [ ]:
@dataclass
class AreaLight:
    """Rectangular area light"""
    center: Vec3
    u: Vec3  # One edge direction
    v: Vec3  # Other edge direction
    color: Color
    intensity: float
    
    def sample_point(self) -> Vec3:
        """Sample random point on light surface"""
        s = random.uniform(-0.5, 0.5)
        t = random.uniform(-0.5, 0.5)
        return self.center + self.u * s + self.v * t
    
    def area(self) -> float:
        """Calculate light area"""
        return self.u.cross(self.v).length()

class Material:
    """Material properties"""
    def __init__(self,
                 diffuse: Color = None,
                 specular: Color = None,
                 roughness: float = 0.0,
                 reflectivity: float = 0.0):
        self.diffuse = diffuse if diffuse else Color(0.8, 0.8, 0.8)
        self.specular = specular if specular else Color(1.0, 1.0, 1.0)
        self.roughness = roughness
        self.reflectivity = reflectivity

class Sphere:
    """Sphere primitive"""
    def __init__(self, center: Vec3, radius: float, material: Material):
        self.center = center
        self.radius = radius
        self.material = material
    
    def intersect(self, ray: Ray) -> Optional[float]:
        """Ray-sphere intersection"""
        oc = ray.origin - self.center
        a = ray.direction.dot(ray.direction)
        b = 2.0 * oc.dot(ray.direction)
        c = oc.dot(oc) - self.radius * self.radius
        discriminant = b*b - 4*a*c
        
        if discriminant < 0:
            return None
        
        t = (-b - math.sqrt(discriminant)) / (2.0 * a)
        if t > 0.001:
            return t
        
        t = (-b + math.sqrt(discriminant)) / (2.0 * a)
        if t > 0.001:
            return t
        
        return None
    
    def normal_at(self, point: Vec3) -> Vec3:
        """Get surface normal at point"""
        return (point - self.center).normalize()

def is_shadowed(point: Vec3, light_point: Vec3, objects: List[Sphere]) -> bool:
    """Check if point is shadowed from light_point"""
    direction = light_point - point
    distance = direction.length()
    shadow_ray = Ray(point, direction)
    
    for obj in objects:
        t = obj.intersect(shadow_ray)
        if t is not None and t < distance:
            return True
    return False

def compute_soft_shadow(point: Vec3, normal: Vec3, light: AreaLight, 
                       objects: List[Sphere], num_samples: int = 16) -> float:
    """Compute soft shadow factor (0 = fully shadowed, 1 = fully lit)"""
    visibility = 0.0
    
    for _ in range(num_samples):
        light_point = light.sample_point()
        if not is_shadowed(point, light_point, objects):
            visibility += 1.0
    
    return visibility / num_samples

print("✓ Soft shadow functions loaded")

## 4. Depth of Field

**Problem:** Traditional ray tracing produces perfectly sharp images (infinite depth of field).

**Solution:** Simulate a lens with finite aperture by sampling rays from different lens positions.

### Thin Lens Model

The **thin lens equation** relates object distance $d_o$, image distance $d_i$, and focal length $f$:

$$
\frac{1}{f} = \frac{1}{d_o} + \frac{1}{d_i}
$$

### Ray Generation

1. Sample random point on lens: $\mathbf{p}_{lens} \in \text{disk}(r_{aperture})$
2. Compute focal point: $\mathbf{p}_{focus} = \mathbf{o} + t_{focus} \cdot \mathbf{d}$
3. New ray origin: $\mathbf{o}' = \mathbf{o} + \mathbf{p}_{lens}$
4. New ray direction: $\mathbf{d}' = \mathbf{p}_{focus} - \mathbf{o}'$

### Circle of Confusion

For objects not at focus distance, the blur radius is:

$$
r_{blur} = \frac{A \cdot |d_o - d_{focus}|}{d_o}
$$

where $A$ is the aperture diameter.

## 5. Motion Blur

**Problem:** Objects in motion appear frozen in time.

**Solution:** Sample rays at different times during the shutter interval and interpolate object positions.

### Temporal Sampling

For each ray, sample time $t \in [t_0, t_1]$ uniformly:

$$
t = t_0 + \xi \cdot (t_1 - t_0), \quad \xi \sim U(0,1)
$$

### Object Position Interpolation

Linear motion:
$$
\mathbf{p}(t) = \mathbf{p}_0 + \mathbf{v} \cdot t
$$

where $\mathbf{v}$ is velocity vector.

In [ ]:
class MovingSphere(Sphere):
    """Sphere with linear motion for motion blur"""
    def __init__(self, center0: Vec3, center1: Vec3, 
                 time0: float, time1: float,
                 radius: float, material: Material):
        super().__init__(center0, radius, material)
        self.center0 = center0
        self.center1 = center1
        self.time0 = time0
        self.time1 = time1
    
    def center_at_time(self, time: float) -> Vec3:
        """Get sphere center at given time (linear interpolation)"""
        t = (time - self.time0) / (self.time1 - self.time0)
        return self.center0 + (self.center1 - self.center0) * t
    
    def intersect(self, ray: Ray) -> Optional[float]:
        """Ray-sphere intersection with motion blur"""
        center_at_ray_time = self.center_at_time(ray.time)
        oc = ray.origin - center_at_ray_time
        
        a = ray.direction.dot(ray.direction)
        b = 2.0 * oc.dot(ray.direction)
        c = oc.dot(oc) - self.radius * self.radius
        discriminant = b*b - 4*a*c
        
        if discriminant < 0:
            return None
        
        t = (-b - math.sqrt(discriminant)) / (2.0 * a)
        if t > 0.001:
            return t
        
        t = (-b + math.sqrt(discriminant)) / (2.0 * a)
        if t > 0.001:
            return t
        
        return None
    
    def normal_at(self, point: Vec3, time: float) -> Vec3:
        """Get surface normal at point and time"""
        center = self.center_at_time(time)
        return (point - center).normalize()

print("✓ Motion blur support loaded")

## 6. Glossy Reflections

**Problem:** Perfect mirror reflections are unrealistic for most materials.

**Solution:** Perturb reflection direction based on surface roughness.

### Microfacet Theory

Rough surfaces have microscopic facets oriented randomly. The reflected ray is perturbed:

$$
\mathbf{r}_{glossy} = \mathbf{r}_{perfect} + \alpha \cdot \mathbf{p}
$$

where:
- $\mathbf{r}_{perfect}$ is the perfect mirror reflection
- $\alpha$ is the roughness parameter ($0 = $ mirror, $1 = $ diffuse)
- $\mathbf{p}$ is a random perturbation vector

### Reflection with Roughness

$$
\mathbf{r} = \mathbf{d} - 2(\mathbf{d} \cdot \mathbf{n})\mathbf{n} + \alpha \cdot \mathbf{random}
$$

In [ ]:
def reflect(v: Vec3, n: Vec3) -> Vec3:
    """Perfect mirror reflection"""
    return v - n * (2.0 * v.dot(n))

def reflect_glossy(v: Vec3, n: Vec3, roughness: float) -> Vec3:
    """Glossy reflection with roughness"""
    perfect_reflection = reflect(v, n)
    
    if roughness > 0:
        # Perturb reflection direction
        perturbation = random_in_unit_sphere() * roughness
        return (perfect_reflection + perturbation).normalize()
    
    return perfect_reflection

print("✓ Glossy reflection functions loaded")

## 7. Complete Distribution Ray Tracer

Combining all techniques into a unified renderer.

In [ ]:
class Scene:
    """Scene containing objects and lights"""
    def __init__(self):
        self.objects = []
        self.lights = []
        self.background_color = Color(0.5, 0.7, 1.0)  # Sky blue
    
    def add_object(self, obj):
        self.objects.append(obj)
    
    def add_light(self, light):
        self.lights.append(light)
    
    def intersect(self, ray: Ray) -> Optional[Tuple[float, Sphere]]:
        """Find nearest intersection"""
        closest_t = float('inf')
        hit_object = None
        
        for obj in self.objects:
            t = obj.intersect(ray)
            if t is not None and t < closest_t:
                closest_t = t
                hit_object = obj
        
        if hit_object:
            return (closest_t, hit_object)
        return None

def shade(ray: Ray, hit_point: Vec3, normal: Vec3, material: Material,
          scene: Scene, shadow_samples: int = 8) -> Color:
    """Compute shading with soft shadows"""
    color = Color(0.1, 0.1, 0.1)  # Ambient
    
    for light in scene.lights:
        # Compute soft shadow
        visibility = compute_soft_shadow(hit_point, normal, light, 
                                        scene.objects, shadow_samples)
        
        if visibility > 0:
            # Sample light for diffuse calculation
            light_point = light.sample_point()
            light_dir = (light_point - hit_point).normalize()
            
            # Diffuse
            n_dot_l = max(0.0, normal.dot(light_dir))
            diffuse = material.diffuse * light.color * (light.intensity * n_dot_l * visibility)
            
            color = color + diffuse
    
    return color.clamp(0.0, 1.0)

def trace_ray(ray: Ray, scene: Scene, depth: int, max_depth: int = 5,
             shadow_samples: int = 8) -> Color:
    """Recursive ray tracing with distribution effects"""
    if depth >= max_depth:
        return Color(0, 0, 0)
    
    hit = scene.intersect(ray)
    
    if not hit:
        # Sky gradient
        t = 0.5 * (ray.direction.y + 1.0)
        return Color(1.0, 1.0, 1.0) * (1.0 - t) + scene.background_color * t
    
    t, obj = hit
    hit_point = ray.at(t)
    
    # Get normal (handle moving spheres)
    if isinstance(obj, MovingSphere):
        normal = obj.normal_at(hit_point, ray.time)
    else:
        normal = obj.normal_at(hit_point)
    
    # Local shading
    local_color = shade(ray, hit_point, normal, obj.material, scene, shadow_samples)
    
    # Glossy reflection
    if obj.material.reflectivity > 0:
        reflect_dir = reflect_glossy(-ray.direction, normal, obj.material.roughness)
        reflect_ray = Ray(hit_point + normal * 0.001, reflect_dir, ray.time)
        reflect_color = trace_ray(reflect_ray, scene, depth + 1, max_depth, shadow_samples)
        
        local_color = local_color * (1.0 - obj.material.reflectivity) + \
                     reflect_color * obj.material.reflectivity
    
    return local_color.clamp(0.0, 1.0)

def render(camera: Camera, scene: Scene, width: int, height: int,
          samples_per_pixel: int = 4, max_depth: int = 5,
          shadow_samples: int = 8, use_motion_blur: bool = False) -> np.ndarray:
    """Render scene with distribution ray tracing"""
    image = np.zeros((height, width, 3))
    
    for j in range(height):
        if j % 10 == 0:
            print(f"Scanline {j}/{height}")
        
        for i in range(width):
            color = Color(0, 0, 0)
            
            # Multiple samples per pixel
            for _ in range(samples_per_pixel):
                # Random offset for anti-aliasing
                u = (i + random.random()) / (width - 1)
                v = (j + random.random()) / (height - 1)
                
                # Random time for motion blur
                time = random.random() if use_motion_blur else 0.0
                
                ray = camera.get_ray(u, v, time)
                color = color + trace_ray(ray, scene, 0, max_depth, shadow_samples)
            
            # Average samples
            color = color / samples_per_pixel
            
            # Gamma correction
            color = Color(
                math.sqrt(color.r),
                math.sqrt(color.g),
                math.sqrt(color.b)
            )
            
            image[height - 1 - j, i] = color.to_tuple()
    
    return image

print("✓ Distribution ray tracer loaded")

## Example 1: Anti-Aliasing Comparison

In [ ]:
# Simple scene for anti-aliasing test
scene = Scene()

# Ground plane (large sphere)
ground_material = Material(diffuse=Color(0.5, 0.5, 0.5), roughness=0.0)
scene.add_object(Sphere(Vec3(0, -1000, 0), 1000, ground_material))

# Sphere
sphere_material = Material(diffuse=Color(0.8, 0.3, 0.3), reflectivity=0.3, roughness=0.1)
scene.add_object(Sphere(Vec3(0, 1, 0), 1.0, sphere_material))

# Area light
light = AreaLight(
    center=Vec3(5, 5, 5),
    u=Vec3(1, 0, 0),
    v=Vec3(0, 1, 0),
    color=Color(1, 1, 1),
    intensity=1.0
)
scene.add_light(light)

# Camera without DOF
camera = Camera(
    look_from=Vec3(3, 3, 5),
    look_at=Vec3(0, 1, 0),
    up=Vec3(0, 1, 0),
    vfov=45,
    aspect_ratio=16/9,
    aperture=0.0
)

# Render with 1 sample (no anti-aliasing)
print("Rendering without anti-aliasing (1 sample)...")
image_no_aa = render(camera, scene, 200, 112, samples_per_pixel=1, shadow_samples=1)

# Render with 16 samples (anti-aliasing)
print("Rendering with anti-aliasing (16 samples)...")
image_aa = render(camera, scene, 200, 112, samples_per_pixel=16, shadow_samples=1)

# Display comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
ax1.imshow(image_no_aa)
ax1.set_title('No Anti-Aliasing (1 sample)')
ax1.axis('off')
ax2.imshow(image_aa)
ax2.set_title('With Anti-Aliasing (16 samples)')
ax2.axis('off')
plt.tight_layout()
plt.show()

print("Notice the smoother edges in the anti-aliased image.")

## Example 2: Soft Shadows

In [ ]:
# Scene with multiple spheres for shadow demo
scene_shadows = Scene()

# Ground
scene_shadows.add_object(Sphere(Vec3(0, -1000, 0), 1000, 
                                Material(diffuse=Color(0.5, 0.5, 0.5))))

# Spheres
scene_shadows.add_object(Sphere(Vec3(-2, 1, 0), 1.0, 
                                Material(diffuse=Color(0.8, 0.2, 0.2))))
scene_shadows.add_object(Sphere(Vec3(0, 1, 0), 1.0, 
                                Material(diffuse=Color(0.2, 0.8, 0.2))))
scene_shadows.add_object(Sphere(Vec3(2, 1, 0), 1.0, 
                                Material(diffuse=Color(0.2, 0.2, 0.8))))

# Large area light for soft shadows
light_soft = AreaLight(
    center=Vec3(0, 8, 3),
    u=Vec3(2, 0, 0),
    v=Vec3(0, 0, 2),
    color=Color(1, 1, 1),
    intensity=1.2
)
scene_shadows.add_light(light_soft)

camera_shadows = Camera(
    look_from=Vec3(0, 3, 10),
    look_at=Vec3(0, 1, 0),
    up=Vec3(0, 1, 0),
    vfov=40,
    aspect_ratio=16/9
)

# Render with soft shadows
print("Rendering with soft shadows (16 shadow samples)...")
image_soft = render(camera_shadows, scene_shadows, 320, 180, 
                   samples_per_pixel=4, shadow_samples=16)

plt.figure(figsize=(12, 7))
plt.imshow(image_soft)
plt.title('Soft Shadows from Area Light')
plt.axis('off')
plt.tight_layout()
plt.show()

print("Notice the gradual shadow penumbra (soft edges).")

## Example 3: Depth of Field

In [ ]:
# Scene with spheres at different distances
scene_dof = Scene()

# Ground
scene_dof.add_object(Sphere(Vec3(0, -1000, 0), 1000,
                           Material(diffuse=Color(0.5, 0.5, 0.5))))

# Spheres at different depths
scene_dof.add_object(Sphere(Vec3(-3, 0.5, 2), 0.5,
                           Material(diffuse=Color(0.8, 0.2, 0.2))))
scene_dof.add_object(Sphere(Vec3(0, 0.5, 0), 0.5,
                           Material(diffuse=Color(0.2, 0.8, 0.2))))
scene_dof.add_object(Sphere(Vec3(3, 0.5, -2), 0.5,
                           Material(diffuse=Color(0.2, 0.2, 0.8))))

# Light
light_dof = AreaLight(
    center=Vec3(5, 5, 5),
    u=Vec3(0.5, 0, 0),
    v=Vec3(0, 0.5, 0),
    color=Color(1, 1, 1),
    intensity=1.0
)
scene_dof.add_light(light_dof)

# Camera with depth of field (focus on middle sphere)
camera_dof = Camera(
    look_from=Vec3(0, 2, 8),
    look_at=Vec3(0, 0.5, 0),
    up=Vec3(0, 1, 0),
    vfov=40,
    aspect_ratio=16/9,
    aperture=0.3,
    focus_distance=8.0
)

print("Rendering with depth of field...")
image_dof = render(camera_dof, scene_dof, 320, 180,
                  samples_per_pixel=32, shadow_samples=4)

plt.figure(figsize=(12, 7))
plt.imshow(image_dof)
plt.title('Depth of Field (focused on center sphere)')
plt.axis('off')
plt.tight_layout()
plt.show()

print("Notice the blur on foreground and background spheres.")

## Example 4: Motion Blur

In [ ]:
# Scene with moving sphere
scene_motion = Scene()

# Ground
scene_motion.add_object(Sphere(Vec3(0, -1000, 0), 1000,
                              Material(diffuse=Color(0.5, 0.5, 0.5))))

# Moving sphere (left to right)
moving_sphere = MovingSphere(
    center0=Vec3(-3, 1, 0),
    center1=Vec3(3, 1, 0),
    time0=0.0,
    time1=1.0,
    radius=0.8,
    material=Material(diffuse=Color(0.8, 0.2, 0.2), reflectivity=0.3)
)
scene_motion.add_object(moving_sphere)

# Static sphere for reference
scene_motion.add_object(Sphere(Vec3(0, 0.5, -3), 0.5,
                              Material(diffuse=Color(0.2, 0.2, 0.8))))

# Light
light_motion = AreaLight(
    center=Vec3(0, 8, 5),
    u=Vec3(1, 0, 0),
    v=Vec3(0, 0, 1),
    color=Color(1, 1, 1),
    intensity=1.0
)
scene_motion.add_light(light_motion)

camera_motion = Camera(
    look_from=Vec3(0, 3, 10),
    look_at=Vec3(0, 1, 0),
    up=Vec3(0, 1, 0),
    vfov=40,
    aspect_ratio=16/9
)

print("Rendering with motion blur...")
image_motion = render(camera_motion, scene_motion, 320, 180,
                     samples_per_pixel=32, shadow_samples=4,
                     use_motion_blur=True)

plt.figure(figsize=(12, 7))
plt.imshow(image_motion)
plt.title('Motion Blur (red sphere moving left to right)')
plt.axis('off')
plt.tight_layout()
plt.show()

print("Notice the blur trail on the moving red sphere.")

## Example 5: Glossy Reflections

In [ ]:
# Scene comparing mirror vs glossy reflections
scene_glossy = Scene()

# Ground
scene_glossy.add_object(Sphere(Vec3(0, -1000, 0), 1000,
                              Material(diffuse=Color(0.5, 0.5, 0.5))))

# Perfect mirror (left)
scene_glossy.add_object(Sphere(Vec3(-2, 1, 0), 1.0,
                              Material(diffuse=Color(0.8, 0.8, 0.8),
                                      reflectivity=0.9,
                                      roughness=0.0)))

# Slightly rough (center)
scene_glossy.add_object(Sphere(Vec3(0, 1, 0), 1.0,
                              Material(diffuse=Color(0.8, 0.8, 0.8),
                                      reflectivity=0.9,
                                      roughness=0.1)))

# Very rough (right)
scene_glossy.add_object(Sphere(Vec3(2, 1, 0), 1.0,
                              Material(diffuse=Color(0.8, 0.8, 0.8),
                                      reflectivity=0.9,
                                      roughness=0.3)))

# Colored sphere to reflect
scene_glossy.add_object(Sphere(Vec3(0, 0.5, -3), 0.5,
                              Material(diffuse=Color(0.8, 0.2, 0.2))))

# Light
light_glossy = AreaLight(
    center=Vec3(5, 8, 5),
    u=Vec3(1, 0, 0),
    v=Vec3(0, 0, 1),
    color=Color(1, 1, 1),
    intensity=1.0
)
scene_glossy.add_light(light_glossy)

camera_glossy = Camera(
    look_from=Vec3(0, 3, 8),
    look_at=Vec3(0, 1, 0),
    up=Vec3(0, 1, 0),
    vfov=45,
    aspect_ratio=16/9
)

print("Rendering glossy reflections (roughness: 0.0, 0.1, 0.3)...")
image_glossy = render(camera_glossy, scene_glossy, 320, 180,
                     samples_per_pixel=32, shadow_samples=4)

plt.figure(figsize=(12, 7))
plt.imshow(image_glossy)
plt.title('Glossy Reflections (left=mirror, center=slight rough, right=very rough)')
plt.axis('off')
plt.tight_layout()
plt.show()

print("Notice how reflections become blurrier with increasing roughness.")

## Summary

**Distribution ray tracing** extends basic ray tracing with stochastic sampling to achieve:

1. **Anti-aliasing**: Multiple samples per pixel eliminate jagged edges
2. **Soft shadows**: Area lights with multiple shadow rays create realistic penumbrae
3. **Depth of field**: Lens aperture simulation creates focus effects
4. **Motion blur**: Temporal sampling captures movement
5. **Glossy reflections**: Perturbed reflection rays simulate rough surfaces

### Key Principles

- **Monte Carlo integration**: Average many random samples to approximate integrals
- **Stratified sampling**: Better than pure random for reducing variance
- **Trade-off**: More samples = higher quality but slower rendering
- **Convergence**: Image quality improves as $\sqrt{N}$ where $N$ is sample count

### Performance Considerations

- **Samples per pixel**: 16-64 for good anti-aliasing
- **Shadow samples**: 8-32 for soft shadows
- **Depth of field**: 32-128 samples for smooth bokeh
- **Motion blur**: 16-64 temporal samples

Distribution ray tracing forms the foundation for more advanced techniques like **path tracing** and **bidirectional path tracing**, which we'll explore in the next chapters.